In [1]:
import geopandas as gpd

import sys
sys.path.append('../Mask_RCNN')
import mrcnn.model as modellib

# Autoria
import canopy_detection_4b as orca

In [2]:
logs = '../Mask_RCNN/logs'
config = orca.OrangeCanopyConfig()
dataset = '../datasets/segmentacao_todos/'

In [3]:
model = modellib.MaskRCNN(mode='inference', config=config, model_dir=logs)

Instructions for updating:
Use fn_output_signature instead


In [6]:
weights = 'D:\FelipeSa\greening\weights\coco_DEM/mask_rcnn_oranges_trees_canopy_0050.h5'

model.load_weights(weights, by_name=True)

In [7]:
img_test = gpd.read_file(dataset + '/img_grid.geojson')
ids = img_test.loc[img_test['split_samples'] == 'test','id'].values
len(ids)

342

In [8]:
results = orca.prediction(model, dataset, id_list=ids , verbose=0, first=False, join=True)

Inferincdo copas:   0%|                                                                        | 0/342 [00:00<?, ?it/s]C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
Inferincdo copas: 100%|██████████████████████████████████████████████████████████████| 342/342 [05:09<00:00,  1.10it/s]


Gerando arquivo de saida


C:\Users\Colaborador\greening\PYTHON\hlb_utils.py:638: UserWarning: `keep_geom_type=True` in overlay resulted in 4 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  over = gpd.overlay(pols, pols, how='intersection')
Simplificando - Micro: 100%|████████████████████████████████████████████████████| 10878/10878 [01:03<00:00, 170.57it/s]
C:\Users\Colaborador\greening\PYTHON\hlb_utils.py:638: UserWarning: `keep_geom_type=True` in overlay resulted in 2 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  over = gpd.overlay(pols, pols, how='intersection')
Simplificando - Micro: 100%|████████████████████████████████████████████████████████| 864/864 [00:03<00:00, 264.00it/s]


Concluído!


In [ ]:
results

In [ ]:
hlb_process = orca.hlb_utils.Processing()
path = '../datasets/segmentacao_todos/img_patches/00033.tif'
array, image_tif = hlb_process.readimagetif(path, 'Integer')
result = model.detect([array[:, :, 0:3]], verbose=1)[0]

In [ ]:
import numpy as np
import tensorflow as tf
# Get activations of a few sample layers
activations = model.run_graph([array[:, :, 0:3]], [
    ("input_image",        tf.identity(model.keras_model.get_layer("input_image").output)),
    ("res2c_out",          model.keras_model.get_layer("res2c_out").output),
    ("res3c_out",          model.keras_model.get_layer("res3c_out").output),
    ("res4w_out",          model.keras_model.get_layer("res4w_out").output),  # for resnet100
    ("rpn_bbox",           model.keras_model.get_layer("rpn_bbox").output),
    ("roi",                model.keras_model.get_layer("ROI").output),
])

In [ ]:
display_images(np.transpose(activations["res2c_out"][0,:,:,:100], [2, 0, 1]), cols=5)

In [ ]:
shape_path_aux = '../datasets/inferencia_sobradinho_areasul/results/'

orca.hlb_ultils.join_vectors()

In [ ]:
import os
import numpy as np
import pandas as pd 
from tqdm import tqdm
from time import sleep

centroid_dist = 0.5
limiar_ovelap = 0.5

shape_path_aux = '../datasets/segmentacao_todos/results'

result_paths = []
for filename in os.listdir(shape_path_aux):
    if os.path.splitext(filename)[1].lower() == '.geojson':
        result_paths.append(os.path.join(shape_path_aux, filename))

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols = pols[pols.geometry.is_valid]
pols['geometry'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)
pols = pols.explode('geometry').reset_index(drop=True)
'''
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= centroid_dist]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

# pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(['union', 'centroid'], axis=1, inplace=True)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
'''
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index

In [ ]:
pols.loc[pols.geometry.geom_type != 'Polygon']

In [ ]:
import os
import numpy as np
import pandas as pd 
from tqdm import tqdm
from time import sleep

centroid_dist = 0.5
limiar_ovelap = 0.5

shape_path_aux = '../datasets/segmentacao_todos/results'

result_paths = []
for filename in os.listdir(shape_path_aux):
    if os.path.splitext(filename)[1].lower() == '.geojson':
        result_paths.append(os.path.join(shape_path_aux, filename))

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols = pols[pols.geometry.is_valid]
pols['geometry'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)
pols = pols.explode('geometry').reset_index(drop=True)
'''
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= centroid_dist]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

# pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(['union', 'centroid'], axis=1, inplace=True)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
'''
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index
# pols['centroid'] = pols['geometry'].centroid

for _ in range(2):
    # Gerar intersecoes entre os poligonos encontrados
    # Uso de busca indexada (R-tree) com a função gpd.overlay()
    over = gpd.overlay(pols, pols, how='intersection')
    # Remover poligonos vazios (talvez possa ser dispensado a partir de agora)
    over = over.loc[-over['geometry'].is_empty]
    # Remover intersecoes originadas pela sobreposição da feição com ela mesma
    over = over.loc[over['id_1'] != over['id_2']]
    # .explode().reset_index(drop=True)

    delete_lines = []
    # Barra de progresso #
    with tqdm(total=len(over['id_1'])) as pbar:
        pbar.set_description("Simplificando - Micro")
        sleep(0.1)

        over['area'] = over['geometry'].area
        over = over.sort_values('area')

        # Para cada poligono..
        for i in over['id_1']:
            # intersecções entre o poligono avaliado e todas as outras feições
            intersec = over[over['id_1'] == i]

            # Caso alguma área de intersecao avaliada seja maior que 60% (limiar_overlap) do proprio poligono avaliado
            # ou se poligono avaliado é 3x menor que aquele com o qual existe intersecao, o poligono será excluido]
            if intersec.loc[intersec['geometry'].area >= limiar_ovelap * pols.loc[i, 'geometry'].area].any(axis=None) | \
                    (intersec['geometry'].area.sum() >= limiar_ovelap * pols.loc[i, 'geometry'].area) | \
                    (3 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():

                # Inserido em 18/12 para evitar falha de detecção
                # Não deletar o maior poligono formador da maior intersecao
                maior = intersec.loc[
                    intersec['geometry'].area == max(intersec['geometry'].area), ['id_1', 'id_2']]
                if pols.loc[maior['id_1'], 'geometry'].area.values[0] < \
                        pols.loc[maior['id_2'], 'geometry'].area.values[0]:
                    delete_lines.append(i)

            pbar.update(1)

        # Identificar poligonos a serem apagados
        delete_lines = (list(set(delete_lines)))
    # Fim da barra de progresso #

    pols.drop(labels=delete_lines, axis=0, inplace=True)

pols = pols.explode().reset_index(drop=True)

pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index

pols_filtred = pols.loc[pols['detection_score'] >= 0.99]

In [ ]:
pols.to_file('../datasets/segmentacao_todos/pols.geojson', driver='GeoJSON')

In [ ]:
pols

In [ ]:
over.loc[over['id_2'].isin(pols.index)]

In [ ]:
over1[over1['id_1'] == i]

In [ ]:
pols.index

In [ ]:
pols.loc[maior['id_1'], 'geometry'].area.values[0] < pols.loc[maior['id_2'], 'geometry'].area.values[0]

In [ ]:
pols

In [ ]:
pols.loc[maior['id_1'], 'geometry'].area

In [ ]:
maior['id_1'], 'geometry'.area

In [ ]:
pols.loc[maior['id_1'], 'geometry'].area.values

In [ ]:
pols.loc[maior['id_2'], 'geometry'].area

all_files = [gpd.read_file(i) for i in result_paths]

In [ ]:
len(all_files)

In [ ]:
pols = pd.concat(all_files, axis=0).reset_index(drop=True)

In [ ]:
pols

In [ ]:
pols = pols[pols.geometry.is_valid]
pols

In [ ]:
pols.geometry.is_valid[-pols.geometry.is_valid]

In [ ]:
pols['geometry_n'] = pols.buffer(0.1).buffer(-0.1).simplify(0.03)

In [ ]:
pols['geometry_n'].is_empty.any()

In [ ]:
pols.geometry_n.is_empty[pols.geometry_n.is_empty]

In [ ]:
for i in range(len(all_files)):
    if all_files[i].is_empty.all():
        print(i)

In [ ]:
filtered_files = [gpd.read_file(i) for i in result_paths if gpd.read_file(i)]

In [ ]:
import geopandas as gpd

import sys
sys.path.append('../Mask_RCNN')
import mrcnn.model as modellib

# Autoria
import canopy_detection as orca

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

In [ ]:
logs = '../Mask_RCNN/logs'
config = orca.OrangeCanopyConfig()
dataset = '../datasets/segmentacao_todos/'

In [ ]:
model = modellib.MaskRCNN(mode='inference', config=config, model_dir=logs)

In [ ]:
weights = '../Mask_RCNN/logs/oranges_trees_canopy20211014T2316/mask_rcnn_oranges_trees_canopy_0060.h5'
weights = '..\Mask_RCNN\logs\oranges_trees_canopy20211217T1047\mask_rcnn_oranges_trees_canopy_0050.h5'

model.load_weights(weights, by_name=True)